# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd

data=pd.read_csv('/datasets/data.csv')
print('')
print('Общая информация о предоставленной для анализа таблице:')
print('')
data.info()
print('')
print('Пример данных, содержащихся в таблице:')
print('')
print(data.head())




Общая информация о предоставленной для анализа таблице:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB

Пример данных, содержащихся в таблице:

   children  days_employed  dob_years education  education_id  \
0      

**Вывод:**
Имеем неподготовленные для анализа данные, нуждающиеся в предварительной обработке с целью приведения их в оптимальный вид для последующей работы. Необходимо устранить пропуски, удалить дупликаты, подготовить данные для работы.


## Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
print('Количество пропусков в столбцах таблицы:')
print(data.isnull().sum())
print('')


print('Заполним все пропуски с некорректными значениями значением "Nan" и выведем для примера первые пять строк с пропусками:')
data['days_employed'] = pd.to_numeric(data['days_employed'], errors='coerce')
data['total_income'] = pd.to_numeric(data['total_income'], errors='coerce')
print('')

print(data[data['days_employed'].isna()].head()) 
print('')
print(data[data['total_income'].isna()].head())



Количество пропусков в столбцах таблицы:
children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Заполним все пропуски с некорректными значениями значением "Nan" и выведем для примера первые пять строк с пропусками:

    children  days_employed  dob_years education  education_id  \
12         0            NaN         65   среднее             1   
26         0            NaN         41   среднее             1   
29         0            NaN         63   среднее             1   
41         0            NaN         50   среднее             1   
55         0            NaN         54   среднее             1   

            family_status  family_status_id gender  income_type  debt  \
12       гражданский брак                 1      M    п

**Вывод:**
Были обнаружены 2174 пропуска в столбцах <i>"days_employed"</i>(общий трудовой стаж в днях) и <i>"total_income"</i> (доход). Вероятно, отсутствие этих данных связано с объективными причинами, как отсутствие работы в конкретный момент времени, соответственно - дохода, и отсутствие трудового стажа(студенты, неофициальная работа).  


### Замена типа данных

In [3]:

print('Заменим отрицательные значения стажа работы и количества детей на положительные, а стаж переведем в года.')
print('Выведем несколько строк таблицы для примера:')
data['children']= abs(data['children'])
data['days_employed']= abs(data['days_employed'])
data['days_employed']= data['days_employed'].apply(lambda x: x/365)
print(data.head())

dayemp_meddiana = round(data['days_employed'].median(), 4) 
totalinc_mediana = round(data['total_income'].median(), 4)
print('')
print('С помощью аргумента ".median" вычислили медианные значения для столбцов с пропусками: ')
print('Медианное значение общего трудового стажа = ', dayemp_meddiana)
print('Медианное значение ежемесячного дохода = ', totalinc_mediana)
print('')

print('Заполнили пропуски значениями их медиан. Для проверки выведены несколько строк обрабатываемых столбцов таблицы:')
data['days_employed']= data['days_employed'].fillna(value=dayemp_meddiana)
data['total_income']= data['total_income'].fillna(value=totalinc_mediana)
print('')
print('Ежемесячный доход:')
print(data['total_income'].head())
print('')
print('Общий трудовой стаж, переведенный в года:')
print(data['days_employed'].head()) 

Заменим отрицательные значения стажа работы и количества детей на положительные, а стаж переведем в года.
Выведем несколько строк таблицы для примера:
   children  days_employed  dob_years education  education_id  \
0         1      23.116912         42    высшее             0   
1         1      11.026860         36   среднее             1   
2         0      15.406637         33   Среднее             1   
3         3      11.300677         32   среднее             1   
4         0     932.235814         53   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0      M   сотрудник     0  145885.952297   
3   женат / замужем                 0      M   сотрудник     0  267628.550329   
4  гражданский брак                 1      F   пенсио

### Обработка дубликатов

In [4]:
print('Перед непосредственным поиском дубликатов необходимо перевести все символы строк с категориальными значениями в единый регистр для исключения скрытых дубликатов.')
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
data['income_type'] = data['income_type'].str.lower()
data['purpose'] = data['purpose'].str.lower()
print('Считаем количество дубликатов по всей таблице с использованием аргументов ".duplicated()" и ".sum()": ', data.duplicated().sum())
print('')
print('Далее методом "value_counts()" выводим уникальные значения в значимых для анализа столбцах в порядке убывания частоты встречаемости: ')
print('')
print('Уникальные значения в стобце "Уровень обазования клиента":')
print(data['education'].value_counts()) 
print('')
print('Уникальные значения в стобце "Семейное положение":')
print(data['family_status'].value_counts()) 
print('')
print('Уникальные значения в стобце "Тип занятости":')
print(data['income_type'].value_counts()) 
print('')
print('В стобце "Цель получения кредита" нельзя дать оценку количеству дубликатов в связи с относительной субъективностью информации в графе:')
print(data['purpose'].value_counts()) 




Перед непосредственным поиском дубликатов необходимо перевести все символы строк с категориальными значениями в единый регистр для исключения скрытых дубликатов.
Считаем количество дубликатов по всей таблице с использованием аргументов ".duplicated()" и ".sum()":  71

Далее методом "value_counts()" выводим уникальные значения в значимых для анализа столбцах в порядке убывания частоты встречаемости: 

Уникальные значения в стобце "Уровень обазования клиента":
среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Уникальные значения в стобце "Семейное положение":
женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Уникальные значения в стобце "Тип занятости":
сотрудник          11119
компаньон           5085
пенсионер           3856

**Вывод:** В исходной таблице отсутствуют каке-либо индивидуальные идентификаторы для клиентов, а имеющаяся информация волей случая может совпадать у разных людей по нескольким или по каждому пункту, т е смело такие дубликаты не удалить. В итоге, выходит, что можно работать только с проблемами в нарушении регистра. Исходя из полученных данных, в столбцах с информацией об образовании, семейном положении и типе занятости дубликатов не наблюдается. Однако, в столбце "Цель получения кредита" по смыслу совпадающие причины изложены с использованием разных форм одних и техже слов, а значит, чтобы проанализировать данные этого столбца в дальнейшем, необходимо сгруппировать цели выдачи кредита методом лемматизации.

### Лемматизация

In [5]:
from pymystem3 import Mystem
m = Mystem() 
from collections import Counter
print('Лемматизируем проблемный столбец для сведения однокоренных слов к объединяющим их значениям для формирования групп, необходимых для дальнейшего анализа целей кредита клиентов банка.')
print('Получаем таблицу:')
for i in range (len(data)):
    lemmas = m.lemmatize(data['purpose'][i])

data['purpose_lemma'] = data['purpose'].apply(m.lemmatize)
print(data['purpose_lemma'].head(10))

Лемматизируем проблемный столбец для сведения однокоренных слов к объединяющим их значениям для формирования групп, необходимых для дальнейшего анализа целей кредита клиентов банка.
Получаем таблицу:
0                      [покупка,  , жилье, \n]
1            [приобретение,  , автомобиль, \n]
2                      [покупка,  , жилье, \n]
3         [дополнительный,  , образование, \n]
4                    [сыграть,  , свадьба, \n]
5                      [покупка,  , жилье, \n]
6               [операция,  , с,  , жилье, \n]
7                            [образование, \n]
8          [на,  , проведение,  , свадьба, \n]
9    [покупка,  , жилье,  , для,  , семья, \n]
Name: purpose_lemma, dtype: object


**Вывод:** В ходе лемматизации получили список запросов на получение кредита: недвижимость и сопутствующие расходы; автомобили и сопутствующие расходы; получение образования;  личные цели. Данные запросы не вызывают сомнений в достоверности полученных данных, подходят для дальнейшей работы и других проверок не требуют.

### Категоризация данных

In [6]:
print('С целью выявления зависимости "надежности" клиента банка от наличия  детей выполним категоризацию данных по следующим признакам:')
print('- "Клиенты с долгами без детей" в значении: children == 0 и значении debt = 1')
print('- "Добросовестные клиенты без детей"  в значении: children == 0  и значении debt = 0')
print('- "Клиенты с долгами с детьми" в значении: children >= 1 и значении debt = 1')
print('- "Добросовестные клиенты с детьми"  в значении: children >= 1  и значении debt = 0')
print('Используем функцию для строки с возвратом заданных значений.')
print('')

def children_debt(row):
    children = row['children']
    debt = row['debt']
   
    if  children == 0:
        if debt == 1:
            return 'Клиенты с долгами  без детей'
       
    if  children == 0:
        if debt == 0:
            return 'Добросовестные клиенты  без детей'
        
    if children >= 1:
        if debt == 1:
            return 'Клиенты с долгами c детьми'
        
    if   children >= 1:
        if debt == 0:
            return 'Добросовестные клиенты c детьми'
        
print('Для проверки функции подставляем значения "children" = 0, "debt" = 1.') 

row_values = [0, 1]

row_columns = ['children', 'debt'] 
row = pd.Series(data=row_values, index=row_columns)
 
print('Получаем: "', children_debt(row), '", что соответствует заданным параметрам.') 
print('')
print('Далее добавляем в исходную таблицу колонку, содержащую присвоенную каждому клиенту категорию.')

data['children_return'] = data.apply(children_debt, axis=1)

print('Считаем количество клиентов в каждой категории:')
print(data['children_return'].value_counts())
print('')

 


С целью выявления зависимости "надежности" клиента банка от наличия  детей выполним категоризацию данных по следующим признакам:
- "Клиенты с долгами без детей" в значении: children == 0 и значении debt = 1
- "Добросовестные клиенты без детей"  в значении: children == 0  и значении debt = 0
- "Клиенты с долгами с детьми" в значении: children >= 1 и значении debt = 1
- "Добросовестные клиенты с детьми"  в значении: children >= 1  и значении debt = 0
Используем функцию для строки с возвратом заданных значений.

Для проверки функции подставляем значения "children" = 0, "debt" = 1.
Получаем: " Клиенты с долгами  без детей ", что соответствует заданным параметрам.

Далее добавляем в исходную таблицу колонку, содержащую присвоенную каждому клиенту категорию.
Считаем количество клиентов в каждой категории:
Добросовестные клиенты  без детей    13086
Добросовестные клиенты c детьми       6698
Клиенты с долгами  без детей          1063
Клиенты с долгами c детьми             678
Name: children_re

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
print('Для ответа на поставленный вопрос считаем проценты каждой группы методом "value_counts()" с использованием перевода серии в датафрейм.')
print('Затем пропускаем получившуюся таблицу через цикл, который высчитывает процентное значение каждой категории от общего количества клиентов банка.')
sum_clients= data['children_return']

print('')
print('Получаем следующие значения: ')
sum_clients= data['children_return']
sum_clients_proc=sum_clients.value_counts().to_frame()
for index in sum_clients_proc:
    sum_clients_proc=round(sum_clients_proc['children_return']*100/len(sum_clients), 2)
    
    
print(sum_clients_proc)
print('')
#количество детей
print('Смотрим, как влияет количество детей на добросовестность выплаты кредита.')
print('Для этого сначала смотрим, подходят ли данные в столбце "Количество детей" для анализа. Выводим уникальные значения столбца методом "value_counts()": ')
print(data['children'].value_counts())
print('')
print('Видим, что в столбце присутствует значение количества детей, равное 20. Принимаем это значение за опечатку ввиду иррациональности и заменяем значение методом "replace" на "2".')

data['children']=data['children'].replace(20, 2)

print('Выводим данные для проверки:')

print(data['children'].value_counts())
print('')
print('Далее с помощью цикла формируем новый столбец с данными, разделяющими клиентов на пять групп по количеству детей: ')
print('клиенты без детей, добросовестные клиенты и должники с одним или двумя детьми, многодетные добросовестные клиенты и должники.')

def children_sum(row):
    children_all = row['children']
    debt = row['debt']
   
    if  children_all <= 2:
        if debt == 1:
            return 'Клиенты с долгами, имеющие одного или двух детей'
       
    if  children_all <= 2:
        if debt == 0:
            return 'Добросовестные, имеющие одного или двух детей'
        
    if children_all >2 :
        if debt == 1:
            return 'Клиенты с долгами многодетные'
        
    if   children_all >2 :
        if debt == 0:
            return 'Добросовестные клиенты многодетные'
    else:
         return 'Нет детей'

data['children_all_return'] = data.apply(children_sum, axis=1)

print('')
print('Формируем новую переменную, в которую включаем только детных клиентов банка. ')
clients_children_sum = data.loc[data['children_all_return'] != 'Нет детей']
print('Далее методами "value_counts()" и "to_frame()" формируем таблицу, отражающую количество представителей каждой группы детных клиентов. Выводим на экран для наглядности:')



print(clients_children_sum['children_all_return'].value_counts())

print('')
clients_children_proc=clients_children_sum['children_all_return'].value_counts().to_frame()
print('Далее считаем проценты по аналогии с ранее использованным методом. Выводим данные:')
for all_child in clients_children_proc:
    clients_children_proc=round(100*clients_children_proc['children_all_return']/len(clients_children_sum), 2)
    
    
print(clients_children_proc)



Для ответа на поставленный вопрос считаем проценты каждой группы методом "value_counts()" с использованием перевода серии в датафрейм.
Затем пропускаем получившуюся таблицу через цикл, который высчитывает процентное значение каждой категории от общего количества клиентов банка.

Получаем следующие значения: 
Добросовестные клиенты  без детей    60.79
Добросовестные клиенты c детьми      31.12
Клиенты с долгами  без детей          4.94
Клиенты с долгами c детьми            3.15
Name: children_return, dtype: float64

Смотрим, как влияет количество детей на добросовестность выплаты кредита.
Для этого сначала смотрим, подходят ли данные в столбце "Количество детей" для анализа. Выводим уникальные значения столбца методом "value_counts()": 
0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

Видим, что в столбце присутствует значение количества детей, равное 20. Принимаем это значение за опечатку ввиду иррациональности и заменяем

**Вывод:**
На основе результатов  анализа можно сделать вывод, что должников с детьми меньше, чем без детей (4.94% должников без детей против 3.15% детных должников). А значит, наличие детей у желающего получить кредит может являться дополнительным положительным фактором при принятии решения об одобрении. Также, исходя из полученных результатов, многодетные клиенты банка являются более надежными заемщиками, чем с одним ребенком или двумя детьми(7.94% малодетных против 0.14% многодетных должников).

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:
print('Методом "value_counts()" выводим данные о возможных семейных статусах клиентов:')
print(data['family_status'].value_counts())

print('Далее с помощью функций разбиваем клиентов на группы в соответствии с данными о семейном положении и данными о наличии/отсутствии долга.')
def family_status_and_debt(row):
    family_status = row['family_status']
    debt = row['debt']
   
    if family_status == 'женат / замужем':
        if debt == 1:
            return 'Клиент с долгами в браке'
       
    if family_status == 'женат / замужем':
        if debt == 0:
            return 'Добросовестный клиент в браке'
        
        
    if family_status == 'гражданский брак':
        if debt == 1:
            return 'Клиент с долгами в гражданском браке'
       
    if family_status == 'гражданский брак':
        if debt == 0:
            return 'Добросовестный клиент в гражданском браке'
        
        
    if family_status == 'не женат / не замужем':
        if debt == 1:
            return 'Клиент с долгами не женат/не замужем'
       
    if family_status == 'не женат / не замужем':
        if debt == 0:
            return 'Добросовестный клиент не женат/не замужем'
               
    if family_status == 'в разводе':
        if debt == 1:
            return 'Клиент с долгами в разводе'
       
    if family_status == 'в разводе':
        if debt == 0:
            return 'Добросовестный клиент в разводе'
        
        
    if family_status == 'вдовец / вдова':
        if debt == 1:
            return 'Клиент с долгами вдовец/вдова'
       
    if family_status == 'вдовец / вдова':
        if debt == 0:
            return 'Добросовестный клиент вдовец/вдова'
print('')       
print('Для проверки функции подставляем значения "family_status" = "вдовец / вдова", "debt" = 1.')
row_values = [ 'вдовец / вдова', 1 ]

row_columns = ['family_status', 'debt'] 
row = pd.Series(data=row_values, index=row_columns)
 
print('Получаем: "', family_status_and_debt(row), '", что соответствует предложенным параметрам.') 
print('')
print('Далее создаем новый столбец в исходной таблице и заполняем его в соответствии с параметрами клиентов. Методом ".value_counts()" получаем сводку:')
data['family_status_return'] = data.apply(family_status_and_debt, axis=1)
print(data['family_status_return'].value_counts())

print('')
print('Считаем проценты каждой категории от общего количества клиентов:')
status=data['family_status_return']
status_sum=status.value_counts().to_frame()
for stat in status_sum:
    status_sum=round(100*status_sum['family_status_return']/len(status), 2)
    
print(status_sum)
print('')
print('Далее формируем таблицу, состоящую из должников и выводим для наглядности:')
clients_status_def = data.loc[data['debt'] != 0]

print(clients_status_def['family_status_return'].value_counts())

status_def_proc=clients_status_def['family_status_return'].value_counts().to_frame()
for stat in status_def_proc:
    status_def_proc=round(100*status_def_proc['family_status_return']/len(clients_status_def), 2)
print('')
print('Далее выводим процентное распределение по семейному положению по отношению к общему количеству должников:')
print(status_def_proc)




print('')
print('Формируем таблицу, состоящую из добросовестных клиентов:')
clients_status_good = data.loc[data['debt'] != 1]

print(clients_status_good['family_status_return'].value_counts())
print('')
status_good_proc=clients_status_good['family_status_return'].value_counts().to_frame()
for stat in status_good_proc:
    status_good_proc=round(100*status_good_proc['family_status_return']/len(clients_status_good), 2)

print('Далее выводим процентное распределение по семейному положению по отношению к общему количеству клиентов без долгов:')
print(status_good_proc)

debt_from_family = pd.DataFrame()
debt_from_family['debt_status'] = clients_status_def.groupby('family_status')['debt']. count()
debt_from_family['all_status'] = data.groupby('family_status')['debt'].count()
debt_from_family['procent_debt'] = round( debt_from_family['debt_status']*100/debt_from_family['all_status'], 1)
print(debt_from_family)


Методом "value_counts()" выводим данные о возможных семейных статусах клиентов:
женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64
Далее с помощью функций разбиваем клиентов на группы в соответствии с данными о семейном положении и данными о наличии/отсутствии долга.

Для проверки функции подставляем значения "family_status" = "вдовец / вдова", "debt" = 1.
Получаем: " Клиент с долгами вдовец/вдова ", что соответствует предложенным параметрам.

Далее создаем новый столбец в исходной таблице и заполняем его в соответствии с параметрами клиентов. Методом ".value_counts()" получаем сводку:
Добросовестный клиент в браке                11449
Добросовестный клиент в гражданском браке     3789
Добросовестный клиент не женат/не замужем     2539
Добросовестный клиент в разводе               1110
Клиент с долгами в браке                       931
Добросовестный

**Вывод:**
Основная масса заемщиков банка - добросовестные плательщики, состоящие в браке(53.19%). В структуре должников наибольший процент составляют так же клиенты, состоящие в браке(53.48%), вторые по количеству - клиенты, имеющие неузаконенные отношения(22.29%). Среди добросовестных клиентов так ж преобладают семейные(57.87%) и состоящиев гражданском браке(19.15%).
Но, те кто развелись или овдовели чаще платят в срок, чем люди в браке.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [9]:
import statistics 
from statistics import mean


print('Так как значения заработной платы в таблице условны, чтобы сгруппировать клиентов по уровню дохода следует определить некую границу, разделяющую клиентов на две группы - с доходом выше среднего и с доходом ниже среднего.')
print('Вычисляем медианное и среднее арифметическое значение заработной платы клиентов из исходной таблицы.')
mediana_total_income = round(statistics.median(data['total_income']))
mean_total_income = round(mean(data['total_income']))
print('Средняя заработная плата=', mean_total_income)
print('Медианная заработная плата=', mediana_total_income )
print('Существенного отличия между средней и медианной зарплатами нет, решено разделить клиентов на группы с доходом выше среднего и ниже среднего полученного значения.')



print('Далее с помощью функций разбиваем клиентов на группы в соответствии с данными о доходе и данными о наличии/отсутствии долга.')
def income_and_debt(row):
    total_income_class = row['total_income']
    debt = row['debt']
   
    if total_income_class >= 165159:
        if debt == 1:
            return 'Клиент с долгами с доходом выше среднего'
       
    if total_income_class >= 165159:
        if debt == 0:
            return 'Добросовестный клиент с доходом выше среднего'

    if total_income_class < 165159:
        if debt == 1:
            return 'Клиент с долгами с доходом ниже среднего'
       
    if total_income_class < 165159:
        if debt == 0:
            return 'Добросовестный клиент с доходом ниже среднего'

        
print('')       
print('Для проверки функции подставляем значения "total_income" = 100000, "debt" = 1.')
row_values = [ 100000, 1 ]

row_columns = ['total_income', 'debt'] 
row = pd.Series(data=row_values, index=row_columns)
 
print('Получаем: "', income_and_debt(row), '", что соответствует предложенным параметрам.')         
        
print()
print('')
print('Создаем новый столбец в исходной таблице и заполняем его в соответствии с полученной классификацией. Далее формируем таблицу с процентным соотношением категорий:')
data['income_and_debt'] = data.apply(income_and_debt, axis=1)


income_proc=data['income_and_debt'].value_counts().to_frame()
for stat in income_proc:
    income_proc['proc']=round(100*income_proc['income_and_debt']/len(data), 2)


print(income_proc)


Так как значения заработной платы в таблице условны, чтобы сгруппировать клиентов по уровню дохода следует определить некую границу, разделяющую клиентов на две группы - с доходом выше среднего и с доходом ниже среднего.
Вычисляем медианное и среднее арифметическое значение заработной платы клиентов из исходной таблицы.
Средняя заработная плата= 165159
Медианная заработная плата= 145018
Существенного отличия между средней и медианной зарплатами нет, решено разделить клиентов на группы с доходом выше среднего и ниже среднего полученного значения.
Далее с помощью функций разбиваем клиентов на группы в соответствии с данными о доходе и данными о наличии/отсутствии долга.

Для проверки функции подставляем значения "total_income" = 100000, "debt" = 1.
Получаем: " Клиент с долгами с доходом ниже среднего ", что соответствует предложенным параметрам.


Создаем новый столбец в исходной таблице и заполняем его в соответствии с полученной классификацией. Далее формируем таблицу с процентным соот

**Вывод:** Основная масса получивших кредит имеют доход ниже среднего значения в выборке. Очевидных закономерностей при делении клиентов на группы по уровню дохода и наличию или отсутствию долга не выявлено. И среди клиентов с низким доходом, и среди клиентов с высоким доходом есть должники.

- Как разные цели кредита влияют на его возврат в срок?

In [10]:

print('Для ответа на поставленный вопрос требуется сначала разбить клиентов на категории в соответствии с целью кредита.')
print('Для этого столбец "purpose" кредита нужно лемматизировать и на основе его создать новый столбец с категориями.')


nedvig = ['жиль', 'недвиж']
wedding = ['свадьб']
auto = ['автомоб']
education = ['образование']

data['purpose_category'] = 0
def categoryes(list_of_words, category):
  join = ''.join(list_of_words)
  index = data[data['purpose'].str.lower().str.contains(join)].index.to_list()
  for i in index:
    data.loc[i, 'purpose_category'] = category
  return data


categoryes(wedding, 'свадьба')
categoryes(nedvig, 'недвижимость')
categoryes(auto, 'авто')
categoryes(education, 'образование')

data['purpose_category']=data['purpose_category'].replace(0, "недвижимость") #без этого выходит 0 вместо недвижимости, не знаю, как исправить

print('Выводим общее количество клиентов в каждой категории: ')
print(data['purpose_category'].value_counts())
print('')

print('Формируем таблицу, где  "sum_purpose" это количество должников в каждой категории, "count_purpose" - общее количество клиентов в данной категории, "result_purpose" - процент должников в каждой категории.')
debt_from_purpose_category = pd.DataFrame()
debt_from_purpose_category['sum_purpose'] = data.groupby('purpose_category')['debt'].sum()
debt_from_purpose_category['count_purpose'] = data.groupby('purpose_category')['debt'].count()
debt_from_purpose_category['result_purpose'] = round(debt_from_purpose_category['sum_purpose']*100 / debt_from_purpose_category['count_purpose'], 1) 
print(debt_from_purpose_category.sort_values('result_purpose', ascending = False))









Для ответа на поставленный вопрос требуется сначала разбить клиентов на категории в соответствии с целью кредита.
Для этого столбец "purpose" кредита нужно лемматизировать и на основе его создать новый столбец с категориями.
Выводим общее количество клиентов в каждой категории: 
недвижимость    12156
авто             4315
образование      2706
свадьба          2348
Name: purpose_category, dtype: int64

Формируем таблицу, где  "sum_purpose" это количество должников в каждой категории, "count_purpose" - общее количество клиентов в данной категории, "result_purpose" - процент должников в каждой категории.
                  sum_purpose  count_purpose  result_purpose
purpose_category                                            
авто                      403           4315             9.3
образование               236           2706             8.7
свадьба                   186           2348             7.9
недвижимость              916          12156             7.5


**Вывод:** Наиболее востребованным является кредит с целью покупки недвижимости. Так же, по получившейся таблице можно сделать вывод, что чаще формируют долги клиенты, взявшие кредит с целью получения образования и покупки автомобиля. 

## Шаг 4. Общий вывод

   Целью исследования стал поиск ответов на вопросы, есть ли зависимость между наличием детей, уровнем дохода, семейным положением клиентов банка и возвратом ими кредита в срок и как разные цели кредита влияют на добросовестность заемщиков?
   
   В начале была произведена подготовка данных к дальнейшей работе. Были заполнены пропуски соответствующими медианными значениями, была проанализирована вероятность дублирующих друг друга строк в данной таблице, так же методом лемматизации был получен список запросов на получение кредита: недвижимость и сопутствующие расходы; автомобили и сопутствующие расходы; получение образования; личные цели. 
   
Далее в соответствии с целями исследования данные были разбиты на различные группы и отсортированы. Исходя из полученных величин были сделаны следующие выводы: семейные люди являются более надежными заемщиками, чем одинокие или состоящие в неузаконенных отношениях клиенты, а так же, процент должников среди овдовевших клиентов и клиентво в разводе ниже. Так же, клиенты с детьми являются более надежными заемщиками, причем, многодетные реже допускают просрочки по оплате. Чаще берут кредит люди с достатком ниже среднего по выборке. Однако, зависимости добросовестности клиента  от уровня дохода не выявлено. Если говорить о целях кредита клиентов, преобладает кредит на покупку недвижимости. Больше всего просрочек у клиентов, взявших кредит на образование и покупку автомобиля. 

   Исходя из выводов можно сформулировать следующие рекомендации банку: оправдана более высокая лояльность в одобрении кредита семейным людям, людям с детьми, а так же вдовым. Так же, оправданно особое внимание к развитию кредитования в сфере недвижимости. Так же, с учетом высокого процента прострочек у групп, взявших кредит на образование и покупку автомобиля, возможно, стоит разработать более гибкие программы для поддержки и повышения спроса или расширить исследование для выявления дополнительных факторов в этих группах, влияющих на добросовестность, с целью отсева наименее благонадежных желающих оформить кредит. 